In [ ]:
from __future__ import annotations

import time
from collections.abc import Callable, Sequence
import matplotlib.pyplot as plt

from tytan import symbols_list, Compile

import numpy as np
from scipy.optimize import minimize
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import QAOAAnsatz
from qiskit_aer import AerSimulator

from qmlant.models.vqe import (
    HamiltonianConverter, IsingConverter, QAOAMixer, circuit_to_einsum_expectation
)
from qmlant.neural_networks.estimator_tn import EstimatorTN

## QUBO

In [ ]:
q = symbols_list(4, "q{}")
C = (q[0] + q[1] + q[2]) ** 2 + \
    q[3] ** 2 + \
    (q[0] + q[1] + q[2]) ** 2 + \
    q[0] ** 2 + \
    q[3] ** 2 + \
    (q[1] + q[2]) ** 2 + \
    q[0] ** 2 + \
    q[3] ** 2 + \
    (q[1] + q[2] + q[3]) ** 2

Q, offset = Compile(C).get_qubo()
ising_dict, additional_offset = IsingConverter(Q).get_ising()
hamiltonian, coefficients = HamiltonianConverter(ising_dict).get_hamiltonian()

## QAOA Quantum Circuit

In [ ]:
def make_initial_state_circuit():
    qc = QuantumCircuit(4)
    qc.h(0)
    qc.cx(0, 1)
    qc.x(0)
    qc.h(2)
    qc.cx(2, 3)
    qc.x(2)
    return qc

def make_placeholder_circuit(
    ising_dict: dict[tuple[str] | tuple[str, str], float],
    n_reps: int = 1,
    insert_barrier: bool = ...,
) -> tuple[QuantumCircuit, Callable[[Sequence[float] | np.ndarray], dict[str, float]]] | int:
    param_names = []

    betas = ParameterVector("β", n_reps)
    beta_idx = iter(range(n_reps))

    def bi():
        return next(beta_idx)

    gammas = ParameterVector("γ", n_reps)
    gamma_idx = iter(range(n_reps))

    def gi():
        return next(gamma_idx)

    qc = make_initial_state_circuit()

    if insert_barrier:
        qc.barrier()

    for _ in range(n_reps):
        # H_P
        gamma = gammas[gi()]
        param_names.append(gamma.name)

        for k in ising_dict:
            if len(k) == 1:
                left = k[0]
                ln = int(left[1:])
                qc.rz(gamma, ln)
            elif len(k) == 2:
                left, right = k  # type: ignore
                ln = int(left[1:])
                rn = int(right[1:])
                assert ln <= rn
                qc.rzz(gamma, ln, rn)
            else:
                raise ValueError(f"len(k) = {len(k)} must be one or two.")

        if insert_barrier:
            qc.barrier()

        # H_M
        beta = betas[bi()]
        param_names.append(beta.name)

        qc.rxx(beta, 0, 1)
        qc.ryy(beta, 0, 1)
        qc.rxx(beta, 2, 3)
        qc.ryy(beta, 2, 3)

        if insert_barrier:
            qc.barrier()

    return qc

In [ ]:
n_reps = 2

qc = make_placeholder_circuit(ising_dict, n_reps=n_reps)
qc.draw()

In [ ]:
expr, operands, pname2locs = circuit_to_einsum_expectation(
    qc,
    hamiltonian,
    coefficients,
    qaoa_mixer=QAOAMixer.XY_MIXER,
)

## Optimization

In [ ]:
losses = []
count = 0

estimator = EstimatorTN(pname2locs, expr, operands)

def compute_expectation_tn(params, *args):
    global count
    (estimator,) = args
    time_start = time.time()
    energy = estimator.forward(params)
    if count % 10 == 0:
        print(f"[{count}] {energy} (elapsed={round(time.time() - time_start, 3)}s)")
    count += 1

    losses.append(energy)

    return energy

In [ ]:
%%time

rng = np.random.default_rng(42)
init = rng.random(qc.num_parameters) * 2*np.pi

result = minimize(
    compute_expectation_tn,
    init,
    args=(estimator,),
    method="COBYLA",
    options={
        "maxiter": 100
    },
)

print(result.message)
print(f"opt value={round(result.fun, 3)}")

## Validate Results

In [ ]:
%%time

qubit_op = SparsePauliOp(
    [ham[::-1] for ham in hamiltonian],
    coefficients,
)

mixer = SparsePauliOp(["XXII", "YYII", "IIXX", "IIYY"], [1/2, 1/2, 1/2, 1/2])

ansatz = QAOAAnsatz(
    cost_operator=qubit_op,
    reps=n_reps,
    initial_state=make_initial_state_circuit(),
    mixer_operator=mixer,
    name='QAOA',
    flatten=None,
)

mapping = operands["make_pname2theta"](result.x)
parameter2value = {param: mapping[param.name] for param in ansatz.parameters}
opt_ansatz = ansatz.bind_parameters(parameter2value)
opt_ansatz.measure_all()

sim = AerSimulator(device="GPU", method="tensor_network")
t_qc = transpile(opt_ansatz, backend=sim)
shots = 1024*50
counts = sim.run(t_qc, shots=shots).result().get_counts()

In [ ]:
for k, n in sorted(counts.items(), key=lambda k_v: -k_v[1]):
    state = k[::-1]  # reverse order
    print(state, n, f"({round(n*100/shots, 2)}%)")

## Visualization

In [ ]:
plt.figure()
x = np.arange(0, len(losses), 1)
plt.plot(x, losses, color="blue")
plt.show()